In [ ]:
from src.grid.make_periodic_grid import make_periodic_grid
import params.PARAMS_ion_acoustic_waves as params
import jax.numpy as jnp

#jnp.set_printoptions(threshold=jnp.inf)

charge = params.charge
Ns = params.Ns
Lx = params.Lx
Lv_s = params.Lv_s
Nx = params.Nx
Nv = params.Nv
finis = params.fini

# Initialize grids and distribution functions
grids = []
for s in range(Ns):
    grid = make_periodic_grid(Lx, Lv_s[s], Nx, Nv)
    grid_methode = "spline"
    grids.append(grid)
    
fs = jnp.zeros((Nx, Nv, Ns))
for s in range(Ns):
    #print(finis)
    fini = finis[s]                                                  #access the distribution function in the params for the s-th species
    #print(fs[:, :, s].shape)
    #print(fini(grids[s]['X'], grids[s]['V'])[511,254:])
    #print(grids[s]['X'], grids[s]['V'])
    fs = fs.at[:, :, s].set(fini(grids[s]['X'], grids[s]['V']).T)
    
#print(fs[:, :, 0])

rho = jnp.zeros(grids[0]["Nx"])  # Initialize rho with zeros
for s in range(Ns):
    rho += charge[s] * jnp.sum(fs[:, :, s], axis=1) * grids[s]['dv']
#print(rho.shape)
#print(rho.max(), rho.min())

kx = (2 * jnp.pi / Lx) * jnp.fft.fftshift(jnp.arange(-Nx//2, Nx//2))
K2 = kx**2
K2 = K2.at[0].set(1)

#print(rho)
b = jnp.fft.fft(rho)
#print(b)
phi_fft = -b / K2
phi_fft = phi_fft.at[0].set(0)  # Set mean to zero

# Compute electric field E = -dphi/dx
dphi_dx_h = 1j * phi_fft * kx
#print(dphi_dx_h.shape, dphi_dx_h.max(), dphi_dx_h.min(), dphi_dx_h)
    
Efield = -jnp.real(jnp.fft.ifft(dphi_dx_h))
print(Efield)

0 [-4.99995947e-01 -4.99845386e-01 -4.99393702e-01 -4.98641133e-01
 -4.97588277e-01 -4.96235847e-01 -4.94584680e-01 -4.92634892e-01
 -4.90388870e-01 -4.87846971e-01 -4.85011697e-01 -4.81884241e-01
 -4.78466392e-01 -4.74760294e-01 -4.70768213e-01 -4.66492653e-01
 -4.61936116e-01 -4.57100987e-01 -4.51990843e-01 -4.46608424e-01
 -4.40956950e-01 -4.35039997e-01 -4.28860664e-01 -4.22423363e-01
 -4.15731072e-01 -4.08788800e-01 -4.01600003e-01 -3.94169569e-01
 -3.86501789e-01 -3.78600955e-01 -3.70472193e-01 -3.62120032e-01
 -3.53550196e-01 -3.44766617e-01 -3.35776210e-01 -3.26583028e-01
 -3.17193389e-01 -3.07612538e-01 -2.97846437e-01 -2.87901044e-01
 -2.77781844e-01 -2.67495751e-01 -2.57048368e-01 -2.46446252e-01
 -2.35695481e-01 -2.24802613e-01 -2.13774920e-01 -2.02618003e-01
 -1.91339016e-01 -1.79944873e-01 -1.68442249e-01 -1.56838298e-01
 -1.45139813e-01 -1.33353829e-01 -1.21487737e-01 -1.09548211e-01
 -9.75427628e-02 -8.54785442e-02 -7.33627081e-02 -6.12030029e-02
 -4.90062237e-02 -3.677

'\nkx = (2 * jnp.pi / Lx) * jnp.fft.fftshift(jnp.arange(-Nx//2, Nx//2))\nK2 = kx**2\nK2 = K2.at[0].set(1)\n\n#print(rho)\nb = jnp.fft.fft(rho)\n#print(b)\nphi_fft = -b / K2\nphi_fft = phi_fft.at[0].set(0)  # Set mean to zero\n\n# Compute electric field E = -dphi/dx\ndphi_dx_h = 1j * phi_fft * kx\n#print(dphi_dx_h.shape, dphi_dx_h.max(), dphi_dx_h.min(), dphi_dx_h)\n    \nEfield = -jnp.real(jnp.fft.ifft(dphi_dx_h))\nprint(Efield)'